In [5]:
import pandas as pd

archivo_legal = r"C:\Users\juan.vermejo\Documents\CPNO\Resumen de casos_legal.xlsx"


In [21]:

df = pd.read_excel(archivo_legal, sheet_name='Denuncia Legal')

df = df[df['TIPO DE CLIENTE'] == 'Comercios']

df['Fecha de Hallazgo CPNO'] = pd.to_datetime(df['Fecha de Hallazgo CPNO'], errors='coerce').dt.strftime('%m.%Y').replace('NaT', None)

# Ordenar el DataFrame por 'id' y 'fecha' (de más reciente a más antigua)
df = df.sort_values(by=['Cuenta Contrato', 'Fecha de Hallazgo CPNO'], ascending=[True, False])

# Eliminar duplicados, conservando la primera ocurrencia (que es la más reciente)
df = df.drop_duplicates(subset='Cuenta Contrato', keep='first')


# Convertir 'Cuenta Contrato' a int, asignar None si falla
df['Cuenta Contrato'] = pd.to_numeric(df['Cuenta Contrato'], errors='coerce').astype('Int64')

# Eliminar filas donde 'Cuenta Contrato' es None para asegurar la integridad de los datos
df.dropna(subset=['Cuenta Contrato'], inplace=True)
df['nros'] = df['COD  CPNO'].str.extract(r'CPNO-(\d+)')
# Convertir los datos limpios a lista para procesamiento posterior
cuentas_legal = df[['Cuenta Contrato', 'nros']].astype(str).values.tolist()

print(cuentas_legal)
cod_cpno = ""

for cc in cuentas_legal:
    if int(cc[0]) == 296010:
        cod_cpno = cc[1]

if cod_cpno == "":
    print(f'Afectación económica {296010} no está en base legal')

[['108583', '02602'], ['108915', '00558'], ['114477', '03616'], ['115204', '03617'], ['118443', '00131'], ['119203', '00529'], ['119207', '00528'], ['132420', '00199'], ['135890', '03876'], ['146013', '04439'], ['148607', '00806'], ['154061', '01245'], ['158087', '02684'], ['160739', '01207'], ['163610', '02307'], ['165688', '02691'], ['166010', '01232'], ['170195', '00921'], ['175121', '00060'], ['181739', '01181'], ['198671', '03671'], ['201908', '02524'], ['204876', '01077'], ['223314', '02676'], ['226434', '02107'], ['227340', '02671'], ['240390', '01876'], ['251913', '02268'], ['262865', '02570'], ['265187', '03863'], ['267949', '03861'], ['268459', '02282'], ['270518', '02548'], ['274077', '02089'], ['278570', '00429'], ['288586', '01009'], ['296010', '00773'], ['299756', '00166'], ['301586', '01922'], ['313315', '02788'], ['316106', '02125'], ['317358', '03694'], ['320145', '02097'], ['320991', '02096'], ['340713', '00321'], ['343902', '04817'], ['344275', '00421'], ['350317', '

In [28]:
import os
import xlwings as xw

# Ruta de la carpeta con los archivos Excel
carpeta_excel = r"C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Actualizados"

# Itera sobre todos los archivos en la carpeta
for archivo in os.listdir(carpeta_excel):
    if archivo.endswith(".xlsx"):  # Filtra solo los archivos Excel
        ruta_excel = os.path.join(carpeta_excel, archivo)
        ruta_pdf = ruta_excel.replace(".xlsx", ".pdf")  # Genera el nombre del PDF en la misma carpeta

        try:
            # Abre el archivo de Excel
            wb = xw.Book(ruta_excel)

            # Selecciona la hoja "HojaLegal"
            hoja = wb.sheets['HojaLegal']

            # Exporta la hoja como PDF
            hoja.api.ExportAsFixedFormat(0, ruta_pdf)

            print(f"Exportación a PDF realizada con éxito: {ruta_pdf}")

        except Exception as e:
            print(f"Ocurrió un error con el archivo {archivo}: {e}")

        finally:
            # Cierra el libro de Excel sin guardar cambios
            if 'wb' in locals():
                wb.close()


Exportación a PDF realizada con éxito: C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Actualizados\853944 - 11.2023 - Informes CPNO.pdf


In [17]:
import os
import pandas as pd
from datetime import datetime

# Ruta de la carpeta con los archivos Excel
carpeta_excel = r'C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Comedores Populares'

# Diccionario de mapeo de encabezados equivalentes
mapeo_columnas = {
    "GIRO": "Clase de PS",
    "TARIFA SAP": "Tarifa (AGOSTO)",
    "Fecha Alta": "Fecha Habilitacion",
    "CLIENTE": "Nombres y Apellidos"
}

# Función para normalizar encabezados de fechas
def normalizar_encabezado_fecha(columna):
    try:
        # Intentar convertir el encabezado en una fecha
        fecha = pd.to_datetime(columna, dayfirst=True)
        # Si es convertible, devolverla en formato "dd/mm/yyyy"
        return fecha.strftime("%d/%m/%Y")
    except (ValueError, TypeError):
        # Si no es una fecha, devolver el encabezado original
        return columna

# Inicializa un DataFrame vacío para almacenar toda la información
df_final = pd.DataFrame()

# Itera sobre los archivos de la carpeta
for archivo in os.listdir(carpeta_excel):
    if archivo.endswith('.xlsx'):  # Filtra solo archivos .xlsx
        ruta_archivo = os.path.join(carpeta_excel, archivo)
        
        # Extraer la "Cuenta Contrato" del nombre del archivo
        cuenta_contrato = archivo.split(' - ')[0]
        
        # Lee el archivo Excel en la hoja 'BD' con encabezados en la fila 2 (índice 1 en pandas)
        df = pd.read_excel(ruta_archivo, sheet_name='BD', header=1)
        
        # Renombrar las columnas según el mapeo definido
        df = df.rename(columns=mapeo_columnas)
        
        # Normalizar los encabezados que sean fechas
        df.columns = [normalizar_encabezado_fecha(col) for col in df.columns]
        
        # Filtrar la fila donde la columna "CC" coincida con la "Cuenta Contrato"
        fila_filtrada = df[df['CC'] == int(cuenta_contrato)]
        
        # Verificar si se encontró la fila
        if not fila_filtrada.empty:
            # Concatenar la fila filtrada con el DataFrame final
            df_final = pd.concat([df_final, fila_filtrada], ignore_index=True)
        else:
            print(f"No se encontró la Cuenta Contrato {cuenta_contrato} en el archivo {archivo}")

# Ruta donde se guardará el DataFrame final
ruta_salida = r'C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Resultados\Mst - lecturas cluster comedores (Inf Originales).xlsx'

# Guardar el DataFrame final en un archivo Excel
df_final.to_excel(ruta_salida, index=False)

# Confirmación final
print(f"Archivo guardado exitosamente en: {ruta_salida}")


Archivo guardado exitosamente en: C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Resultados\Mst - lecturas cluster comedores (Inf Originales).xlsx


In [6]:
import os
import xlwings as xw
import datetime
from pathlib import Path
import pandas as pd
import argparse

archivo_prueba = r"C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Comedores Populares\5184642 - 07.2024 - Informes CPNO.xlsx"
archivo_plantilla = r"C:\Users\juan.vermejo\Documents\CPNO\Plantilla - Informes CPNO.xlsx"

def obtener_valores_y_rango(archivo_excel):
    try:
        app = xw.App(visible=False)  # Ejecutar Excel en segundo plano
        wb = app.books.open(archivo_excel)
        hoja = wb.sheets['Hoja de Calculos']

        valor_total_consumo = None
        fila_item = None
        fila_total_m3h = None
        muestra_tipica_valores = None
        tipo_de_calculo = None
        cuenta_contrato = None
        rango_celdas = None
        ultimo_valor = None
        tiempo_trabajo = None
        dias_trabajo_por_mes = None
        medidor_referencia_sap = None

        cuenta_contrato = hoja.range('C2').value
        print(f"Valor de 'N° Cuenta Contrato' (C2): {cuenta_contrato}")

        for cell in hoja.range('B1:B1000'):
            if cell.value in ["TOTAL CONSUMO RECUPERADO", "TOTAL CONSUMO RELIQUIDACIÓN"]:
                valor_total_consumo = hoja.range(f'D{cell.row}').value
                print(f"Valor de 'TOTAL CONSUMO': {valor_total_consumo}")
            elif cell.value == "Item":
                fila_item = cell.row
            elif cell.value == "Total m3/h":
                fila_total_m3h = cell.row
            elif cell.value == "Muestra Tipica":
                muestra_tipica_valores = (hoja.range(f'C{cell.row}').value, hoja.range(f'D{cell.row}').value)
                print(f"Valores de 'Muestra Tipica': C{cell.row}={muestra_tipica_valores[0]}, D{cell.row}={muestra_tipica_valores[1]}")
            elif cell.value == "Tiempo de trabajo (h)":
                tiempo_trabajo = hoja.range(f'F{cell.row}').value
                print(f"Valor de 'Tiempo de trabajo (h)': {tiempo_trabajo}")
            elif cell.value == "Días de Trabajo por Mes":
                dias_trabajo_por_mes = hoja.range(f'F{cell.row}').value
                print(f"Valor de 'Días de Trabajo por Mes': {dias_trabajo_por_mes}")

        if fila_item is None:
            raise ValueError("No se encontró 'Item' en la columna B.")
        if fila_total_m3h is None:
            raise ValueError("No se encontró 'Total m3/h' en la columna B.")

        rango_celdas = hoja.range(f'C{fila_item + 2}:D{fila_total_m3h - 1}').value
        print(f"Rango de celdas desde 'Item' hasta 'Total m3/h': {rango_celdas}")

        for cell in hoja.range('B2:Z2'):
            if cell.value == "Tipo de Calculo":
                tipo_de_calculo = hoja.range(f'{cell.offset(0, 1).address}').value
                print(f"Valor de 'Tipo de Calculo': {tipo_de_calculo}")
            elif cell.value == "Medidor Referencia SAP":
                medidor_referencia_sap = hoja.range(f'{cell.offset(0, 1).address}').value
                print(f"Valor de 'Medidor Referencia SAP': {medidor_referencia_sap}")

        sheet_names = [sheet.name.strip() for sheet in wb.sheets]
        if 'BDDetalle' in sheet_names:
            hoja_detalle = wb.sheets['BDDetalle']
            valores_columna_a = hoja_detalle.range('A1:A1000').value
            
            for valor in reversed(valores_columna_a):
                if valor not in (None, '', 'Null'):
                    ultimo_valor = valor
                    break

            print(f"Último valor en la columna A de 'BDDetalle': {ultimo_valor}")
        else:
            print(f"La hoja 'BDDetalle' no existe en este archivo. Hojas disponibles: {sheet_names}")

    except Exception as e:
        print(f"Error: {e}")
    finally:
        try:
            wb.close()
            app.quit()
        except UnboundLocalError:
            pass

    return (valor_total_consumo, rango_celdas, muestra_tipica_valores, tipo_de_calculo, cuenta_contrato, tiempo_trabajo, dias_trabajo_por_mes, ultimo_valor, medidor_referencia_sap)




         CC Fecha Habilitacion  \
0   1059786         2019-02-15   
1   1249485         2019-08-14   
2   1261213         2019-09-30   
3   1785553         2022-02-23   
4   1785555         2022-03-03   
5   1812584         2022-01-28   
6   1820049         2016-07-19   
7   1834929         2022-03-23   
8   1834960         2022-01-14   
9   1834990         2022-01-11   
10  1849966         2022-03-19   
11  1937696         2022-03-26   
12   260997         2013-10-09   
13   290209         2013-12-16   
14   290397         2013-12-18   
15  3018842         2022-06-08   
16   338798         2014-05-12   
17   381412         2014-09-25   
18   397821         2015-02-12   
19   403900         2015-03-27   
20   408576         2015-03-12   
21  5080446         2023-01-06   
22  5102296         2023-02-10   
23  5138296         2023-01-10   
24  5144946         2023-01-03   
25  5148458         2023-01-12   
26   517087         2016-11-25   
27  5174425         2022-12-13   
28  5184642   

In [20]:
import pandas as pd

# Ruta del archivo .txt
ruta_txt = r"C:\Users\juan.vermejo\Downloads\CARTERA_GC_08092024.txt"

# Leer el archivo .txt en un DataFrame (ajusta el delimitador según tu tabla)
df = pd.read_csv(ruta_txt, delimiter='\t', encoding='ISO-8859-1')

# Exportar a un archivo Excel
ruta_excel = r"C:\Users\juan.vermejo\Downloads\CARTERA_GC_08092024..xlsx"
df.to_excel(ruta_excel, index=False)  # Eliminar el índice si no lo necesitas


C:\Users\juan.vermejo\AppData\Local\Temp\ipykernel_6248\675895222.py:7: DtypeWarning: Columns (8,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_txt, delimiter='\t', encoding='ISO-8859-1')


In [23]:
import pandas as pd

# Ruta del primer archivo (ya extraído previamente)
archivo_plantilla = r"C:\Users\juan.vermejo\Documents\CPNO\Plantilla - Informes CPNO.xlsx"

# Ruta del segundo archivo
archivo_prueba = r"C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Comedores Populares\5184642 - 07.2024 - Informes CPNO.xlsx"

# Equivalencias de columnas
mapeo_columnas = {
    "GIRO": "Clase de PS",
    "TARIFA SAP": "Tarifa (AGOSTO)",
    "Fecha Alta": "Fecha Habilitacion",
    "CLIENTE": "Nombres y Apellidos"
}

# Leer el archivo plantilla (anteriormente)
df_plantilla = pd.read_excel(archivo_plantilla, sheet_name='BD', header=1)

# Leer el archivo de prueba
df_prueba = pd.read_excel(archivo_prueba, sheet_name='BD', header=1)

# Normalización de encabezados en el archivo de prueba
def normalizar_columnas(columnas, mapeo):
    columnas_normalizadas = []
    for col in columnas:
        # Si la columna está en el mapeo, reemplazarla por su equivalente
        if col in mapeo:
            columnas_normalizadas.append(mapeo[col])
        else:
            columnas_normalizadas.append(col)
    return columnas_normalizadas

# Aplicar el mapeo de equivalencias de columnas al DataFrame del archivo de prueba
df_prueba.columns = normalizar_columnas(df_prueba.columns, mapeo_columnas)

# Mostrar los encabezados del segundo archivo después de aplicar el mapeo
print("Encabezados normalizados del segundo archivo:")
print(df_prueba.columns)

# Mostrar una parte del DataFrame de prueba para verificar
print("\nContenido del segundo archivo:")
print(df_prueba.head())


Encabezados normalizados del segundo archivo:
Index(['CC', 'Fecha Habilitacion', 'Nombres y Apellidos', 'Clase de PS',
       'N° Medidor', '1/01/2019', '1/02/2019', '1/03/2019', '1/04/2019',
       '1/05/2019', '1/06/2019', '1/07/2019', '1/08/2019', '1/09/2019',
       '1/10/2019', '1/11/2019', '1/12/2019', '1/01/2020', '1/02/2020',
       '1/03/2020', '1/04/2020', '1/05/2020', '1/06/2020', '1/07/2020',
       '1/08/2020', '1/09/2020', '1/10/2020', '1/11/2020', '1/12/2020',
       '1/01/2021', '1/02/2021', '1/03/2021', '1/04/2021', '1/05/2021',
       '1/06/2021', '1/07/2021', '1/08/2021', '1/09/2021', '1/10/2021',
       '1/11/2021', '1/12/2021', '1/01/2022', '1/02/2022', '1/03/2022',
       '1/04/2022', '1/05/2022', '1/06/2022', '1/07/2022', '1/08/2022',
       '1/09/2022', '1/10/2022', '1/11/2022', '1/12/2022', '1/01/2023',
       '1/02/2023', '1/03/2023', '1/04/2023', '1/05/2023', '1/06/2023',
       '1/07/2023', '1/08/2023', '1/09/2023', '1/10/2023', '1/11/2023',
       '1/12/202

In [13]:
## BOLSA GERALDINE

import pandas as pd

# Ruta del archivo Excel original y la nueva ruta donde se guardará el archivo
archivo_excel = r"C:\Users\juan.vermejo\Documents\CPNO\BolsaLibre.xlsx"
nuevo_archivo = r"C:\Users\juan.vermejo\Documents\CPNO\BolsaLibre_Asignado.xlsx"

# Leer el archivo Excel usando pandas
df = pd.read_excel(archivo_excel, sheet_name="Cruce")

# Filtrar los registros que tienen FALSO en la columna "Cerrado con Deposito?"
df_filtrado = df[df['Cerrado con Deposito?'] == False]
# df_filtrado = df[df['Esta en agenda IBR?'] == False]
df_filtrado = df[df['Estatus Gestión Comercial'] != "Se firmó acuerdo"]


# Aplicar más filtros:
# 1. Eliminar registros con valores vacíos en "InformesElaborados.Ingreso CPNO esperado"
# 2. Excluir registros donde "InformesElaborados.Ingreso CPNO esperado" sea igual a 590 o 1180
df_filtrado = df_filtrado[df_filtrado['InformesElaborados.Ingreso CPNO esperado'].notna()]  # Filtrar valores no vacíos
df_filtrado = df_filtrado[~df_filtrado['InformesElaborados.Ingreso CPNO esperado'].isin([590, 1180])]  # Excluir 590 y 1180

# Verificar si el DataFrame no está vacío después del filtrado
if df_filtrado.empty:
    print("No se encontraron registros que cumplan con los criterios de filtrado.")
else:
    # Nombres de las personas para asignar clientes
    personas = ['Jhonny Llontop', 'Manuel Huambachano', 'Fraciny Minaya', 'Jackeline Ferro']

    # Ordenar los clientes por el aporte dinerario "InformesElaborados.Ingreso CPNO esperado" de mayor a menor
    df_filtrado = df_filtrado.sort_values(by='InformesElaborados.Ingreso CPNO esperado', ascending=False).reset_index(drop=True)

    # Número de personas y número total de clientes
    num_personas = len(personas)
    num_clientes = len(df_filtrado)

    # Inicializar listas para almacenar el total de aportes de cada persona
    aportes_personas = {persona: 0 for persona in personas}
    asignaciones = []  # Lista para guardar las asignaciones de cada cliente

    # Distribuir los clientes de manera equitativa pero balanceando el dinero
    for i in range(num_clientes):
        # Calcular qué persona tiene actualmente el menor total de aportes
        persona_asignada = min(aportes_personas, key=aportes_personas.get)

        # Asignar el cliente a esa persona
        asignaciones.append(persona_asignada)

        # Sumar el aporte del cliente al total acumulado de la persona asignada
        aportes_personas[persona_asignada] += df_filtrado.loc[i, 'InformesElaborados.Ingreso CPNO esperado']

    # Agregar la columna 'Persona' con las asignaciones al DataFrame
    df_filtrado['Persona'] = asignaciones

    # Guardar el DataFrame con la nueva columna en un archivo Excel nuevo
    df_filtrado.to_excel(nuevo_archivo, index=False)

    print(f"Asignación completada y archivo guardado en: {nuevo_archivo}")


c:\Users\juan.vermejo\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Cell Y158 is marked as a date but the serial value 964157005 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():
c:\Users\juan.vermejo\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Cell W252 is marked as a date but the serial value 3003554 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


Asignación completada y archivo guardado en: C:\Users\juan.vermejo\Documents\CPNO\BolsaLibre_Asignado.xlsx


In [11]:
import os
import pandas as pd

# Ruta de la carpeta con los archivos
folder_path = r'C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Actualizados\Legal'
# Ruta donde se guardará el archivo consolidado
output_file = r'C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Actualizados\Mst - Consolidado actualizacion legal2.xlsx'

# DataFrame para almacenar los datos
consolidated_df = pd.DataFrame()

# Iterar sobre los archivos en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):  # Solo procesar archivos Excel
        file_path = os.path.join(folder_path, file_name)

        # Leer la hoja 'BDPercy' y saltar las primeras 2 filas (los encabezados están en la fila 3)
        df = pd.read_excel(file_path, sheet_name='BDPercy', header=2)
        
        # Concatenar los datos al DataFrame consolidado
        consolidated_df = pd.concat([consolidated_df, df], ignore_index=True)

# Guardar el DataFrame consolidado en un nuevo archivo Excel
consolidated_df.to_excel(output_file, index=False)
print(f'Datos guardados en: {output_file}')


Datos guardados en: C:\Users\juan.vermejo\Documents\CPNO\Pruebas\Masivo\Actualizados\Mst - Consolidado actualizacion legal2.xlsx
